#### Ejercicio de modularización de código
Este ejercicio presenta un tema que es marginal al meollo del curso, ya que es puro Python. Hasta aquí el desarrollo de los scripts ha sido lineal, es decir, secuencias de sentencias que se ejecutan linealmente, excepto por algún bucle. Este procedimiento se adapta muy bien al esquema de celdas en Jupyter Notebook. Un estilo de programación más profesional, sin embargo, suele propender a la modularización organizando las sentencias como funciones. Una programacion modular ayuda a reciclar codigo y acelerar el desarrollo de aplicaciones similares en el futuro. En particular, una organización con funciones es indispensable cuando se emplean tableros de control como Streamlit o Gradio. 

Entonces, el ejercicio consiste en que reescriban el script que se da abajo empleando funciones. Construyan funciones para la carga de textos, para la partición, y para la indexación. Intenten que las funciones sean autónomas, es decir, que los parámetros requeridos les sean transferidos como argumentos en la invocación.<br>
Se ofrece una solución pero no es la única posible.

##### Script sin modularizar

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq

mixtral = ChatGroq(model="mixtral-8x7b-32768")
loader = TextLoader('./directorio/ConstitucionArgentina.txt',
                   encoding='UTF-8')
docs =loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
final_docs = text_splitter.split_documents(docs)

# Incrustacion y espacio vectorial
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") 
db = FAISS.from_documents(documents=final_docs, embedding=embeddings)

query = "Cuales derechos sostiene la constitucion argentina"
result = db.similarity_search(query)
result[0].page_content

#### Solución - script modularizado
Se incluyen las clases importadas en cada módulo para facilitar las pruebas, pero debieran reunirse al principio.

In [28]:
from langchain_community.document_loaders import TextLoader

def cargar_texto(path, archivo):
    loader = TextLoader(path + archivo, encoding='utf-8')
    docs = loader.load()
    return docs
    
docs = cargar_texto("c:/NuevoChatbot/directorio/", "ConstitucionArgentina.txt")

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def particionador(clase_particionador, docs):
    text_splitter = clase_particionador(chunk_size=1000, chunk_overlap=200)
    docs_split = text_splitter.split_documents(docs)
    return docs_split

particionador(RecursiveCharacterTextSplitter, docs)

[Document(metadata={'source': 'c:/NuevoChatbot/directorio/ConstitucionArgentina.txt'}, page_content='La Constitución de la Nación Argentina, que rige actualmente a la República Argentina, fue aprobada por una asamblea constituyente integrada por representantes de trece provincias, hecha en la ciudad de Santa Fe en el año 1853. El propósito de la Constitución de 1853 fue poner fin al ciclo de las guerras civiles y sentar las bases de la «unión nacional» mediante un régimen republicano y federal. Antes de esta Constitución hubo dos intentos constituyentes en 1819 y 1826, que no prosperaron por carecer de consenso entre las provincias. Con posterioridad otras diez provincias y una ciudad integraron la federación.\n\nEl texto ha sido reformado varias veces, estando vigente la redacción resultante de la reforma de 1994. En varias oportunidades el orden constitucional fue dejado sin efecto por golpes de Estado, el último de los cuales impuso una dictadura que se mantuvo en el poder hasta el 

In [34]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def indexacion(docs, query):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") 
    db = FAISS.from_documents(documents=docs, embedding=embeddings)
    resultado = db.similarity_search(query)
    return resultado

resultado = indexacion(docs, query= "Cuales derechos sostiene la constitucion argentina")
resultado

[Document(metadata={'source': 'c:/NuevoChatbot/directorio/ConstitucionArgentina.txt'}, page_content='La Constitución de la Nación Argentina, que rige actualmente a la República Argentina, fue aprobada por una asamblea constituyente integrada por representantes de trece provincias, hecha en la ciudad de Santa Fe en el año 1853. El propósito de la Constitución de 1853 fue poner fin al ciclo de las guerras civiles y sentar las bases de la «unión nacional» mediante un régimen republicano y federal. Antes de esta Constitución hubo dos intentos constituyentes en 1819 y 1826, que no prosperaron por carecer de consenso entre las provincias. Con posterioridad otras diez provincias y una ciudad integraron la federación.\n\nEl texto ha sido reformado varias veces, estando vigente la redacción resultante de la reforma de 1994. En varias oportunidades el orden constitucional fue dejado sin efecto por golpes de Estado, el último de los cuales impuso una dictadura que se mantuvo en el poder hasta el 